# The PAP algorithm (using the simplified XPP)

Pouchou, J.-L., & Pichoir, F. (1991). Quantitative Analysis of Homogeneous or Stratified Microvolumes Applying the Model “PAP.”

https://sci-hub.ru/10.1007/978-1-4899-2617-3_4



The functions to calculate the steps are put in python files in the PAP_functions folder.

This Jupyter Notebook writes out the equations in LaTeX format. Read the functions in the python files, or write ´function_name??´ in a cell to see the code.

Plotting of the equations, to compare them to the PAP paper, is done in the PAP_plots.ipynb notebook.

In [ ]:
import hyperspy.api as hs
import numpy as np
import plotly.graph_objects as go
import pandas as pd

from PAP_functions.PAP_helper_functions import (theoretical_energy,
    get_C_A_Z_arrays,
    wt2at,
    at2wt,
    calculate_atom_percent)
from PAP_functions.PAP_area_F import (set_m_small, 
    ionization_cross_section_Q, 
    mean_atomic_mass_M, 
    mean_ionzation_potential_J, 
    energy_dependent_terms_f_of_v, 
    energy_loss_dE_drhos, 
    whole_dE_drhos, 
    deceleration_factor_one_over_S, 
    backscattering_factor_R, 
    area_F, 
    mean_atomic_number_Zb)
from PAP_functions.PAP_parameterization import (surface_ionization_phi_zero, 
    average_depth_of_ionization_R_bar, 
    absorption_correction,
    parameterization_phi_of_rhoz)

# F - area of the distribution $\phi(\rho z)$


The area of the $\phi(\rho z)$ curve is the primary intentisty, F.

$ F = \int \limits_{0}^{\inf} \phi(\rho z)d(\rho z) = (R/S) \cdot Q(U)$


We use an approximation for the ionization cross-section, and calculate R and 1/S as below.

___
Sidenote:  $n_A = C_A \cdot \frac{N^0}{A} \cdot \frac{R}{S} $

This is per electron(?). We need $C_A$, $A$, $R$, and $S$. Factors like $N^0$ and the beam current cancel out (shared between the elements in the specimen).

Sidenote end.
___


## $1/S$ - deceleation of electrons

$ 1 / S = \int \limits_{E_0}^{E_l} \frac{Q(U)}{dE/d\rho s} dE$

$Q(U)$ is the ionization cross section. $dE/d\rho s$ is the stopping power of the electrons. This integral is solved analytically with the equations below.



___

### $Q(U)$ - Ionization cross section

(The PAP paper writes "$Q(U)$", where l is the level, $A$ is the atomic number and $E_0$ is the nominal beam energy.However, writing $Q(U)$ is more efficient.)

From the paper: "expressions that were proposed are generally too mathematically complex to allow for an
analytical calculation of the integral (3). A satisfactory way of varying the cross section
with U is obtained with the expression proposed by Hutchins [23]:


$ Q(U) \propto ln(U) / (U^m \cdot E_c^2)$


- with $ U = E/E_c $ as the overvoltage, where $E_c$ is the critical ionization energy.
- m is a constant dependent on the line type (K, L or M)
    - K-lines: m = 0.9, as suggested by Bastin* (1998, PROZA96)
    - L-lines: m = 0.82
    - M-lines: m = 0.78

*Bastin, Dijkstra and Heijligers (1998) improved the m-coefficient in the Q-equation. They have specific numbers for C, N and O, which have low Z.

https://analyticalsciencejournals.onlinelibrary.wiley.com/doi/pdf/10.1002/%28SICI%291097-4539%28199801/02%2927%3A1%3C3%3A%3AAID-XRS227%3E3.0.CO%3B2-L

In [ ]:
# ionization_cross_section_Q??

___

### $dE/d\rho s$, the energy loss function for electrons

Bethe (Ref. 10 in the PAP-paper) have a formula for $dE/d\rho s$, but this is valid for >30 keV. PAP use a formula valid for 1-50 keV:

$dE/d\rho s = -M/J \cdot 1/f(V)$

where


- $M = \sum \limits_{i} \frac{C_i Z_i}{A_i}$


- $ J = \exp(\sum \limits_{i} \frac{C_i Z_i}{A_i} \cdot \ln(J_i)/M)$
    - $J_i = 10^{-3} \cdot Z_i (10.04 + 8.25 \exp(\frac{-Z_i}{11.22}))$


- $ f(V) = \sum \limits_{k=1}^{3} D_k \cdot V^{P_k}$

    - $ V = \frac{E}{J}$
    - $D_1 = 6.6 \cdot 10^{-6}$
    - $P_1 = 0.78$
    - $D_2 = 1.12 \cdot 10^{-5}(1.35 - 0.45 J^2)$
    - $P_2 = 0.1$
    - $D_3 = \frac{2.2 \cdot 10^{-6}}{J}$
    - $P_3 = -(0.5-0.25J)$
    
i.e.

- $f(V) = 6.6 \cdot 10^{-6}\cdot V^{0.78} + \\ 
1.12\cdot 10^{-5}\cdot (1.35 - 0.45 J^2) \cdot V^{0.1} + \\
\frac{2.2 \cdot 10^{-6}}{J} \cdot V^{-(0.5-0.25J)}$


Calculate M, then J, then f(V) and then $dE/d\rho s$


- $dE/d\rho s$ is average energy loss of the electrons, in keV cm^2/g
- $Ci$ is the mass concentration of element i in the sample, in wt%
- $Zi$ is the atomic number of element i, dimensionless
- $Ai$ is the atomic weight of element i, in Da (??)
- $J$ is the mean ionization potential of the sample, in keV
- $Ji$ is the ionization potential of element i, in keV
- $V$ is $E/J$, dimensionless
- $M$ is the mean atomic mass of the sample, in Da (??)

In [ ]:
# energy_loss_dE_drhos??

___

### Analytical solution for $1/S$

$ 1/S = \int \limits_{E_0}^{E_C} Q_C^A(E) \frac{dE}{d\rho s} dE $

Using

$ T_k = 1 + P_k - m $ in 

$ 1/S = \frac{U_0}{V_0 \cdot M} \sum \limits_{k=1}^{3} D_k \cdot (V_0/U_0)^{P_k} \cdot ((T_k)U_0^{T_k} \cdot \ln(U_0)-U_0^{T_k}+1)/T_k^2 $


Which translates to

$1/S = \frac{U_O}{V_0 \cdot M} \cdot \\
(6.6 \cdot 10^{-6} (V_0/U_0)^{0.78} \cdot ((1+0.78-m)U_0^{1+0.78-m} \cdot \ln(U_0)-U_0^{1+0.78-m}+1)/(1+0.78-m)^2) + \\
((1.12 \cdot 10^{-5}(1.35-0.45J^2)) (V_0/U_0)^{0.1} \cdot ((1+0.1-m)U_0^{1+0.1-m} \cdot \ln(U_0)-U_0^{1+0.1-m}+1)/(1+0.1-m)^2) + \\
(2.2 \cdot 10^{-6}/J (V_0/U_0)^{(-0.5 +0.25J)} \cdot ((1+(-0.5 +0.25J)-m)U_0^{1+(-0.5 +0.25J)-m} \cdot \ln(U_0)-U_0^{1+(-0.5 +0.25J)-m}+1)/(1+(-0.5 +0.25J)-m)^2)
$

In [ ]:
# deceleration_factor_one_over_S??

In [ ]:
# Numbers for 1/S for GaSb, using Ga_La line

elements = np.array(['Ga', 'Sb'])
line = 'Ga_La'
e0 = 15  # keV
concentrations = np.array([0.5, 0.5])  # at%

e_c = theoretical_energy(line=line)
u = e0 / e_c
m_small = set_m_small(line=line)

array_C, array_A, array_Z = get_C_A_Z_arrays(elements=elements, 
                                         concentrations=concentrations, concentration_type='at')
q = ionization_cross_section_Q(e0=e0, line=line)
m = mean_atomic_mass_M(array_C=array_C, array_Z=array_Z, array_A=array_A)
j = mean_ionzation_potential_J(array_C=array_C, array_Z=array_Z, array_A=array_A)
f_of_v = energy_dependent_terms_f_of_v(e=e0, j=j)
dE_drhos = energy_loss_dE_drhos(m=m, j=j, f_of_v=f_of_v)
# dE_drhos2 = whole_dE_drhos(array_C=array_C, array_Z=array_Z, array_A=array_A, e0=e0)

s_inverse = deceleration_factor_one_over_S(u0=u, e_c=e_c, j=j, m_big=m, m_small=m_small)


print(f'C [wt%]: {array_C}')
print(f'A [Da]: {array_A}')
print(f'Z: {array_Z}')
print(f'Q: {q:.3e}')
print(f'M [Da?]: {m:.3e}')
print(f'J [keV]: {j:.3e}')
print(f'1/f(V): {1/f_of_v:.3e}')
print(f'dE/drhos [?]: {dE_drhos:.3e}')
# print(f'dE/drhos2: {dE_drhos2:.3e}')
print(f'\n1/S: {s_inverse:.3e}')


___
## R - the backscatter loss factor 

Appendix 1 in PAP.


$ R = 1- \bar{\eta}  \cdot \bar{W} \cdot (1-G(U_0)) $

where

- $\bar{\eta}$ is the mean backscattering coefficient
- $ \bar{\eta} = 1.75 \cdot 10^{-3} \cdot \bar{Z}_b + 0.37(1-\exp(-0.015\bar{Z}_b^{1.3})) $
- $\bar{Z}_b$ is the (weighted) mean atomic number of the backscattered electrons, 
- $\bar{Z}_b = (\sum C_i \cdot Z_i^{0.5})^2$
- $\bar{W}$ might be the work of the backscattered electrons, but I'm not sure
- $ \bar{W} = \bar{E}_r/E_0 = 0.595 + \bar{\eta}/3.7 + \bar{\eta}^{4.55} $

- "The term G(U0) is extracted from the theoretical model of Coulon and Zeller (28)."

    - $ G(U_0) = (U_0 - 1 - (1- \frac{1}{U_0^{1+q}})/(1+q)) / ((2+q)\cdot J(U_0))$

    - $ J(U_0) = 1 + U_0 \cdot (\ln(U_0)-1) $

    - $ q = (2 \bar{W} - 1) / (1 - \bar{W}) $




In [ ]:
# backscattering_factor_R??
# # It is a plot of R vs. Z with different U0 values in the plotting notebook.

In [ ]:
r = backscattering_factor_R(u0=u, array_C=array_C, array_Z=array_Z)
print(f'R: {r:.3e}')

## Calculating F

$ F = (R/S) \cdot Q(U) $

There is a slight contradictive equation:

"In equation (2) it is stated that $n_A = C_A (N^0/A) \cdot Q(U) \cdot F$, and in equation (3) it is stated that $n_A = C_A (N^0/A) (R/S)$.

This implies that $(R/S) = Q(U) \cdot F$.

However, equation (13) states that $F = (R/S) \cdot Q(U)$, which is a contradiction to the previous equations."

It is assumed that Eq. (13) is correct, but the function allows for the user to set Q to 1.


In [ ]:
# area_F(*, array_C: np.ndarray, array_Z: np.ndarray, array_A: np.ndarray, e0: float, line: str, use_q: str = 'divide') -> float:

f = area_F(array_C=array_C, array_Z=array_Z, array_A=array_A, e0=e0, line=line, use_q='divide')
f

In [ ]:
# Testing F on the spectra

In [ ]:
df_whole = pd.read_excel('../sem-eds-qc/results/ZAF_output.xlsx', sheet_name=0)
# drop "i corr" and "k-ratio"
# df = df.drop(columns=['i corr', 'k-ratio'])
df = df_whole[['Group', 'Element', 'Line', 'x', 'kV', 'pA', 'PT',
       'i', 'at_wt',  'AZ at%', 'i: at%',]].copy()
df['i: at%'] = df['i: at%'] * 100
df['AZ at%'] = df['AZ at%'] * 100

df.head(3)

In [ ]:
df['F'] = 0.0
df['F (chi)'] = 0.0
df['f (chi)'] = 0.0
df['Q'] = 0.0

big_F_list = []
big_F_of_chi_list = []
small_f_list = []
q_list = []

for i in range(len(df['x'])):
    elements = df[df['x'] == df['x'][i]]['Element'].values
    at_wt_list = df[df['x'] == df['x'][i]]['at_wt'].values
    e0 = df['kV'][i]
    wt_concentrations = at2wt(at = [0.5, 0.5], atwt=at_wt_list)
    array_C, array_A, array_Z = get_C_A_Z_arrays(elements=elements, concentrations=[0.5, 0.5], concentration_type='at')

    m = mean_atomic_mass_M(array_C=array_C, array_Z=array_Z, array_A=array_A)
    m_small = set_m_small(line=df['Line'][i])
    j = mean_ionzation_potential_J(array_C=array_C, array_Z=array_Z, array_A=array_A)

    e_c = theoretical_energy(df['Line'][i])
    u = e0 / e_c

    q = ionization_cross_section_Q(e0=e0, line = df['Line'][i])
    s_inverse = deceleration_factor_one_over_S(u0=u, e_c=e_c, j=j, m_big=m, m_small=m_small)
    r = backscattering_factor_R(u0=u, array_C=array_C, array_Z=array_Z)
    # big_F_m = area_F(array_C=array_C, array_Z=array_Z, array_A=array_A, e0=df['kV'][i], line=df['Line'][i], use_q='multiply')
    big_F = area_F(array_C=array_C, array_Z=array_Z, array_A=array_A, e0=e0, line=df['Line'][i], use_q='divide')

    f_small = absorption_correction(e0=e0, line=df['Line'][i], elements=elements, wt_concentrations=wt_concentrations,  use_q='ignore')

    big_F_of_chi = f_small * big_F

    big_F_list.append(big_F)
    big_F_of_chi_list.append(big_F_of_chi)
    small_f_list.append(f_small)
    q_list.append(q)
    

df['F'] = big_F_list
df['F (chi)'] = big_F_of_chi_list
df['f (chi)'] = small_f_list
df['Q'] = q_list
    

In [ ]:
df[['Group', 'Line', 'x', 'i', 'i: at%', 'F', 'F (chi)', 'f (chi)', 'Q']].tail(10)

In [ ]:
at_F_list = []
at_f_chi_list = []

for xi in df['x'].unique(): # for each spectrum
    df_xi = df[df['x'] == xi]
    elements = df_xi['Element'].values
    
    i_F = df_xi['i'] / df_xi['F']
    # i_f_chi = df_xi['i'] * ( df_xi['f (chi)'] / df_xi['Q'])
    i_f_chi = df_xi['i'] / ( df_xi['f (chi)'] )


    # wt = i1 / (i1 + i2)
    wt_F = (i_F / i_F.sum()).to_list()
    wt_f_chi = (i_f_chi / i_f_chi.sum()).to_list()

    at_F = calculate_atom_percent(wt_list = wt_F, elements = elements)
    at_f_chi = calculate_atom_percent(wt_list = wt_f_chi, elements = elements)

    at_F_list += list(at_F*100)
    at_f_chi_list += list(at_f_chi*100)


df['F at%'] = at_F_list
df['f (chi) at%'] = at_f_chi_list


### For GaSb, the "Fd" works the best. That is, $F = R/S / Q$

In [ ]:
df[['Group', 'Line', 'i', 'AZ at%', 'i: at%', 'F', 'F at%', 'f (chi) at%', 'Q']]

In [ ]:
df_dev = df[['Group', 'Line', 'AZ at%', 'i: at%', 'F at%', 'f (chi) at%']].copy()
for col in ['AZ at%', 'i: at%', 'F at%', 'f (chi) at%']:
    df_dev[col] = abs(50-df_dev[col])
df_dev.describe()

In [ ]:
# drop 10 and 11
df_dev = df_dev.drop([10, 11])

In [ ]:
df_dev.describe()

# Parameterization of $F(\chi)$

The $\phi(\rho z)$ curve is parameterized through: 

$ F(\chi) = [\phi(0) + B/(b + \chi) - A \cdot b \cdot \epsilon / (b \cdot (1+\epsilon) + \chi)] / (b + \chi) $

The parameters are explained first, then put together in the function at the end.


##  $\phi(0)$ - surface ionization

$\phi(0) = 1 + 3.3 (1-1/U_0^r)* \bar{\eta}^{1.2} $

with $ r = 2 - 2.3 \bar{\eta} $

(The exponent $\bar{\eta}^{1.2}$ kinda looks like $\bar{\eta}^{1 \cdot 2}$, but it is 1.2 that gives the same values as the plot in figure 24.)

In [ ]:
# surface_ionization_phi_zero??

## $\bar{R}$ - mean depth of ionization

Equation 28 in PAP

$ F / \bar{R} = 1 + [X \cdot \ln(1 + Y \cdot (1 - 1/U_0^{0.42}))]/\ln(1 + Y) $

where

- $ X = 1 + 1.3 \ln(\bar{Z}_b) $
- $ Y = 0.2 + \bar{Z}_b/200 $


i.e.

$ \bar{R} = F / (1 + [X \cdot \ln(1 + Y \cdot (1 - 1/U_0^{0.42}))]/\ln(1 + Y)) $

#### adjustments for high $\bar{R}$

"If $ F/\bar{R} < \phi(0) $, impose the condition $ \bar{R} = F/\phi(0) $"

I guess the above adjustment is for numerical stability, but I'm not sure.

In [ ]:
# average_depth_of_ionization_R_bar??

In [ ]:
zb = mean_atomic_number_Zb(array_C=array_C, array_Z=array_Z)
r_bar = average_depth_of_ionization_R_bar(big_f=area_F(array_C=array_C, array_Z=array_Z, array_A=array_A, e0=e0, line=line, use_q='divide'), u=u, zb=zb)
r_bar

## $P$ - initial slope 

equation 29


$ P = g \cdot h^4 \cdot F/\bar{R}^2 $

where

- $ g = 0.22 \ln(4 \bar{Z}_b) \cdot [1 - 2 \exp(-\bar{Z}_b \frac{U_0 - 1}{15})] $

- $ h = 1 - 10(1-\frac{1}{1+ U_0/10})/\bar{Z}_b^2 $


### Comment to the value $ g \cdot h^4 $ in the paper

"If necessary, limit the value $ g \cdot h^4 $ to the value $ 0.9 \cdot b\cdot  \bar{R}^2 \cdot [b - 2 \phi(0)/F] $"

I do not think that is necessary.

In [ ]:
# initial_slope_P??

## Calculate b, a and $\epsilon$

(Wrote b wrong the first time)

$ b = \sqrt{2} \cdot (1 + \sqrt{1 - \bar{R} \cdot \phi(0) / F})/\bar{R} $


$ a = [P + b \cdot (2\phi(0) - b \cdot F)] / [b \cdot F \cdot (2 - b \bar{R}) - \phi(0)] $

$ \epsilon = \frac{a-b}{b}  $

"If necessary, impose on $\epsilon$ a minimum absolute value (e.g. $10^{-6}$), and then assume $ a = b \cdot(1+e) $" (I do not think that is necessary.)

In [ ]:
# factor_small_b??
# factor_small_a??
# factor_epsilon??

## Calculate B and A

$ B = [b^2 \cdot F \cdot (1 + \epsilon) - P - \phi(0) \cdot b \cdot (2+\epsilon) ] / \epsilon $

$ A = [B/b + \phi(0) - b \cdot F] \cdot \frac{1+ \epsilon}{\epsilon} $

## Emergent intensity, through $ F (\chi) $

$ F(\chi) $ is the emergent intensity, as it takes in the absorption correction.

" The fluorescent yield, the weights of the line of interest and the instrumental factors (solid angle and detection efficiency, incident current) are other factors omitted in eq (19) " (PAP p. 38)


$ I_A \propto C_A \cdot Q(U) \cdot F(\chi) $

where


$ \chi = \mu _\rho \cdot \cosec(TOA)$**

$ F(\chi) = [\phi(0) + B/(b + \chi) - A \cdot b \cdot \epsilon / (b \cdot (1+\epsilon) + \chi)] / (b + \chi) $

or

$ F(\chi) = \frac{A}{a+ \chi} + \frac{\phi(0) - A}{b + \chi} + \frac{B}{(b + \chi)^2} $

which are the same.


** Not explicitly stated in the PAP-paper, but assumed based on the reference Love and Scott.


In [ ]:
# absorption_correction??
# 

# plotting $\phi (\rho z)$

$ \phi (\rho z) = A \cdot \exp(- a \cdot (\rho z)) + (B \cdot (\rho z) + \phi(0) - A) \cdot \exp(- b \cdot (\rho z)) $


$ I_A \propto C_A \cdot Q(U) \cdot F(\chi) $

where

- $ C_A $ is the concentration of element A, in wt%
- $ Q(U) $ is the ionization cross section with overvoltage $U$
- $ F(\chi) $ is the emergent intensity

In [ ]:
# parameterization_phi_of_rhoz??

In [ ]:
hei

###### snakkes aldri.

![Image](https://folk.ntnu.no/brynjamm/marathon_dabz.gif)